# ESDA2 Group Project: Investigating how and why Glaciers of the Greenland Ice Sheet have changed in the past few decades.
**Date: 12/11/2025**

Authors:
* Joel Barron
* Eliza Cowie
* Ellie Schneider
* Dongye Li
* Russell Chung

# Section 1: Overview and Methods

## 1.1 Introduction
*(Authors: Ellie, 100%)*

Anthropogenic climate warming is driving a rise in global mean temperature, intensifying cryospheric mass loss in polar regions. Earth hosts two large ice sheets, Greenland and Antarctica, that play a vital role in regulating global sea levels and climate. 

It’s important to investigate Greenland’s response to climate change due to various local and global implications that glacier area loss has. Glaciers form when snow accumulates on land, to the extent that the weight on top compresses deeper snow into ice. Ice sheet dynamics are governed by the balance between accumulation and melt. They advance (terminus extends) when snow accumulation exceeds melt, and retreat (terminus shrinks) when melt is greater than accumulation. NASA calculated that Greenland has lost approximately 5 trillion tons of weight since the early 2000s, and glaciers are now melting six or seven times faster today than 25 years ago (Hall, 2022). Ice loss happens primarily in 2 ways: surface warming and iceberg calving. Surface warming occurs when water melts when air temperature is warm, causing water on top to run into the ocean. Iceberg calving is when ice breaks off the terminus of a glacier because the forward motion makes the terminus unstable influenced by warmer waters (NASA Earthdata, 2024). Both these processes contribute to Greenland’s overall ice mass loss. 

Surface warming is important when considering sea level rise. As glaciers originate on land, when they melt, run-off from land-based ice causes levels to rise as total sea water volume has increased by adding new mass to the ocean. Although, levels do not rise ubiquitously so implications, such as coastal erosion and flooding, vary across regions causing some areas to experience more severe effects than others. 

This freshwater input from melting glaciers can also alter ocean circulation. One big implication is the change in behaviour of the Gulf Stream - a warm current in the North Atlantic that keeps Western Europe climactically warm. As glacial freshwater melts, salinity reduces which can slow or shut down the movement of the Gulf Stream, which could have far-reaching effects on regional temperatures, weather patterns and sea levels.

This report focuses on the Greenland Ice Sheet and its response to ongoing climate change. It aims to investigate how and why glaciers have changed over the past few decades by analysing variations in glacial area, sea surface temperature (SST) and air temperatures along with the interrelated behaviour of these factors. 

## 1.2 Research Questions

Through this report we will investigate the following questions:

- How has the area of Greenland glaciers has changed in recent history?
- How do environmental factors affect glacier area change? Considering the factors:
    * Air temperature 
    * Sea surface temperature (SST) 
- Do our hypothesis line up in extreme cases?
- How might glaciers continue to change in Greenland in the near future? 

## 1.3 Data and Methods
*(Authors: Joel, 100%)*

We used three main types of data for this report: Comma Separated Value (csv), Excel and netCDF. Dealing with csv and Excel data in Python is standard, being able to utilise the Pandas library. Our first data file was ‘glacier_area.xlsx’, an Excel file containing 195 glacier entries, with variables ‘Latitude’, ‘Longitude’ and timestamps ranging from 1972-2022. At each of these timestamps, the table value read the area of the specified glacier at that time in km2. Latitude and longitude were separated as values of a key-value pair in a dictionary of all glaciers. The data frame then had this information dropped, and was transposed to form an area time series for all glaciers.  Our next chunk of data is metdata from six different weather stations around Greenland, each in its own csv file, and an Excel file of details about each station. Each metdata csv file was formatted the same, separated by ‘;’ which was specified when reading in. This contained many different meteorological readings from 1870 to 2024. Each of these files then had to be cleaned. From the data’s directory, we knew that ‘101’ corresponded to air temperature in °C, and columns renamed accordingly. The time of each entry was in separate columns for year, month, day and hour, so we added a new column ‘Date’ containing a datetime object of the appropriate columns. The data was then filtered to a series with the date as the index and ‘airtemp’ as the values. This process was repeated for each station. It is worth noting each of these series contained NA values, most likely from when the station was switched off for maintenance, or another event prevented reading from being taken. These were dealt with accordingly in analysis when taking yearly averages, dropping years with incomplete data. Our final was 'HadISST_SST_northatlantic.nc', a netCDF file containing grid data of sea surface temperature around Greenland. It contained four variables: ‘time’, a 1-dimensional variable of time since 1st Jan 1870; ‘latitude’ and ‘longitude’, each 1-dimensional variables of grid latitudes and longitudes, and ‘SST’, a 3-dimensional variable of time, latitude and longitude, containing the sea surface temperature at that time at that location in °C. This was read using the netCDF4 package. We separated each variable mentioned into a numpy array of appropriate dimension. Variable for time was adjusted to represent a datetime value. The ‘SST’ variable included some odd values, specifically those less than -1.8 (of which some seen at -250 or -1000), as this is the freezing temperature of saltwater. Thus, we masked the SST array so that any value less than -1.8 was mapped to 1.8. While latitude and longitude did not require cleaning, it is worth mentioning that they proved difficult to deal with when plotting, as we had to convert to grid coordinated. All code for cleaning can be seen in appendix A1.1. 

To aid our analysis, we created four unique functions to deal with the data. Firstly, as glacier coordinates did not correspond to exact SST or air temperature readings, a function to find the indices of closest SST (for indexing into the array) and a function to find what stations was closest to the glacier were created and then used throughout the code. Another function was made to get a data frame of concurrent air temperature, SST and glacier area, for a specified glacier was created. This took yearly averages of each variable and did an inner join to form the data frame. This ensured that there were no missing values, very important when conducting regression analysis with the regplot function of the seaborn library. For air temperature, yearly averages only included years which had data for a whole year, so that the result was not affected by the seasonality of temperature, which could skew a result. For example, if a station was switched off for the winter, the mean annual temperature for that year would be higher than it should be. The final function added was for bootstrap sampling a linear regression, taken from the ‘Statistical Thinking in Python (Part 2)’ interactive course (Datacamp.com, 2025). This allowed us to draw slope and intercept bootstrap pairs of the linear regression, utilised in Section 5 of this report. All coded functions can be found in appendix A1.2.

Appendix section A1.3 through A1.7 include all code used to produce plots and figures through the report. Maps were plotted using the Basemap and Cartopy libraries, with data added using matplotlib and seaborn. Lambert conformal conic projection is used extensively, as it does not distort the shape of Greenland too much, when using latitude bounds (58, 78) and longitude bounds (-60, 25), centred on latitude 70, longitude -45, and using standard parallels at latitudes 60 and 80. Section 3.2 includes a map with plate carree projection (code appendix A1.5), chosen to be able to see a wider area around Greenland. Correlation scatter plots through the report (code appendix A1.4 and A1.5) use the inner joined data frame function mentioned above, to get robust data. All other plots are time series plots, created with the matplotlib library, and trends fitted using the numpy library. Other statistics are calculated with the use of the numpy library also. 


# Section 2: How has the area of Greenland glaciers has changed in recent history?
*(Authors: Russell, 80%; Ellie, 10%; Joel, 10%)*

## 2.1 Outline

In this section we will look at the area change of glaciers and trends in Greenland within the plots. It is important that we understand this so that we can see the factors that have contributed to the change.

## 2.2 Analysis

![](greenland_glacier_location.png)

**Figure 1:** Map of the locations of all the Greenland glaciers, each marked by a red dot. 

Figure 1 shows where Greenland glaciers have the highest frequency along the south-east coast and are much sparser along the west/north-west coast. These location variations are highly influenced by both climactic and topographic factors. We can see that most of the glaciers tend to be in locations near the sea, because of factors such as, topography, ocean-ice interactions and calving. Examples of glaciers with close proximity to each other are Jakobshavn, Russell, Maniitsoq and Helheim. Locations where the glaciers are further apart included Steenstrup, Petermann and Sif.

![](percentage_total_area_change.png)

**Figure 2:** This graph shows the percentage of total area change of all the Greenland glaciers from 1972-2022, where 0.1% is 1317.42$km^2$. 

From Figure 2 we can see there are clear fluctuations which are likely caused by variations in temperatures between summer and winter as they are steady fluctuations with similar amplitudes that look to occur annually. The time period from 1972 up until 1985 shows relatively minimal, steady percentage compared to the rest of the behaviour. There are 3 obvious phases of percentage loss, where there are 3 short, rapid episodes of glacier retreat around 1985 (minor but the first noticeable acceleration), 2003 (start of rapid area percentage decline) and 2012 (major area loss). These could be caused by potential significant temperature changes and various excessive climate interactions, such as the summer heatwave in 2012 that caused record-breaking melt to Greenland where an estimated 97% of the ice sheet surface experienced melt on July 11 and 12, just 2 days (NOAA Climate.gov, 2012). From looking at the graph, the overall trend is that percentage area loss will continue to accelerate. This highlights that as the climate gets more extreme, we will see a greater percentage increase in the total area change due to the impact of ice melting faster than before. This is suggested by the gradient becoming slightly steeper after every episode of rapid area decrease, meaning more of the glaciers are melting over a shorter period of time. A total of 4725.17km2,  ~ 0.36% of the area of all the glaciers in Greenland has been lost in just 50 years.

![](individual_glacier_area_change.png)

**Figure 3:** Plot showing area of the individual glaciers with the solid red line representing the mean of the individual glacier area change and the dotted line representing the mean loss in 2022. 

In Figure 3, we can see from the plot that the mean of the individual glacier change is relatively resistant to major change, especially between 1972 to 2000. After 2000 the mean dips slightly. The area change in $km^2$ for most glaciers doesn’t change much between 1972- 1985, with any change being hard to see by eye, highlighting the fact that the glaciers were slow to retreat in the beginning. Post 2000, we see certain outlying glaciers begin to decline in area rapidly, losing hundreds of square kilometres in area. Unsurprisingly, mean area declines as well, to a loss of 24.23$km^2$ by 2022, equivalent to around 3,500 football pitches.  

## 2.3 Discussion

Based on the plots and graphs, we can see that there are many glaciers that have retreated very quickly with the most notable ones being  Umiammakku Isbrae and Rink Isbrae. Many factors can be contributed to the glacier area change such as surface melting, increased submarine melting, accelerated ice flow, calving and albedo. The rising temperatures cause the ice sheets to melt particularly in the summer months. Another factor is that increased submarine melting which are caused by fjords reaching the base of the marine terminating glaciers.  Accelerated ice flow also affects the glacier area as the retreat from the glacier base results in the glacier flowing faster towards the sea (Boyall,2024).In particular, calving plays a significant role in the area change as this is a process in where the glacier will accelerate and reach the coast. As this process happens, many icebergs retreat from ice sheets which in turn the ice is flowing towards the sea. In addition to this, the process creates a positive climate feedback loop in where the increased input of meltwater also can lead to glacier retreat of ice flow (King et al., 2020). The last main factor is the albedo effect. Where darker surfaces emit more radiation due to the reduced reflectivity and as a result have lower albedo. The darker surfaces will absorb more solar radiation accelerating the warming and melting which links very well to the cause of the glacier area change in Greenland.  


# Section 3: How do environmental factors affect glacier area change?

When looking at the factors causing glaciers to behave this way, it is important to consider both changes in air temperature and sea surface temperature, as air temperature increase is the primary driver of surface melt and warmer oceans are causing destabilisation of termini of marine-terminating glaciers, promoting iceberg calving. 

## **3.1 Air Temperature**

*(Authors: Ivy, 100%)*

## 3.1.1 Outline

This section shows that long-term warming in the region, as shown by rising local air temperatures and backed up by records from nearby stations, is linked to a clear drop in glacier area.

## 3.1.2 Analysis

![](annual_mean_temp_432000.png)

**Figure 4:** Annual mean air temperature (°C) at station 432000, 1955–2024, showing strong year-to-year variability and an overall warming trend.

The annual mean air temperature at station 432000 changes a lot from year to year, as seen in Figure 4. The time-series figure shows a series of warm and cool periods that switch back and forth, but the overall trend goes up from the beginning to the end of the record. Approximate reading suggests that annual temperature increased from around −14°C in the 1970s to about −11°C in recent decades. This shows that a long-term warming signal is present despite short-term variability.

![](airtemp_area_regression.png)

**Figure 5:** Negative relationship between annual mean air temperature and glacier area at Rink Isbræ, with higher temperature associated with smaller glacier extent.

![](zachariae_isstrom_concurrent_plot.png)

**Figure 6:** Both plots show data from the glacier Zachariae Isstrom. The top plot shows Glacier Area each year from 1970-2022. The bottom plot shows an increasing trend in Air Temperature over the same timeseries. 

The scatterplot in Figure 5 show how the area of a glacier and the average annual temperature are related. Points are spread out in a band that goes down, with higher temperatures matching smaller glacier areas. The time-series plot (Figure 6) makes it clear that the area of the glacier has been getting smaller since 1972. The decline is moderate from the 1970s to the 1990s, but it speeds up a lot after about 2005. The area of glaciers has gone down in the last ten years. The patterns that happen at the same time as the rise in annual mean temperature suggest that the loss of glacier area is closely linked to warming in the atmosphere. The visible slope of the point shows that the area of the glacier is very sensitive to changes in the local weather because the temperature values vary so much from year to year.

Two consistent themes can be seen in the data: regional air temperatures increasing, and years with warmer weather have smaller glacier areas. The comparison of temperature and glacier area shows that they are related in the opposite way: years with higher temperatures also have smaller glacier areas. These results suggest a shared physical interpretation: Glacier has diminished in area due to prolonged atmospheric warming at this location.

## 3.1.3 Discussion

The temperature record from station 432000 shows a clear and long-lasting warming trend. The annual time-series curve goes up over the course of the observation period. This means that the regional atmosphere has been getting warmer over the course of several decades. This behaviour aligns with trends observed in extensive high-altitude regions experiencing significant warming (Zhao et al., 2024).

The significant decrease in Zachariae Isstrøm after ~2005 shows that it is very sensitive to changes in the balance of surface energy. As the air temperature rises, the melt rate increases mainly because of stronger sensible heat fluxes and a longer ablation season. Warmer summers also make it harder to keep old snow: when the old snow layer can't freeze anymore, meltwater runs off more easily, which means more mass is lost. This change to runoff-dominated melt makes it harder for the glacier to hold onto surface meltwater and speeds up thinning. Also, having more meltwater at the bed can make basal lubrication better, make sliding easier, and cause dynamic thinning in addition to surface ablation.

The temperature-area scatterplot (Figure 5) shows how glaciers react directly. The points are grouped so that higher yearly temperatures are linked to smaller glacier areas. The data point structure indicates that glacier area is responsive to fluctuations in local air temperature; as warmer conditions become increasingly prevalent, the glacier occupies a more restricted low-area range. This interpretation is physically sound, as elevated air temperatures enhance melting energy and diminish the seasonal ice accumulation at the surface. Similar temperature-related mass loss behaviour is observed in North American glaciers (Williamson et al., 2025). 

The plotted outputs show that significant glacier-area loss happens when warming lasts for a long time. The speed at which the retreat has sped up since about 2005 is especially interesting because it matches times when temperatures were higher than normal. These results are in line with what we would expect physically and are similar to what has been reported in other studies, where increased atmospheric warming has caused rapid outlet-glacier mass loss (Williamson et al., 2025). The results show how sensitive Greenland's tidewater glaciers are to change in the climate and how important continued warming is in determining how much ice will be lost in the future.


## **3.2 Sea Surface Temperature** 
*(Authors: Eliza, 100%)*

## 3.2.1 Outline

This section talks about Sea Surface Temperatures (SST) around Greenland and goes into depth of one glacier in specific, a marine terminating glacier Rink Isbrae. We know SST has a direct relationship with air temperature as they both exchange energy with each other, and has an inverse relationship with Glacier Area. We show how it has a negative effect and in fact, causes glaciers to retreat. Reasons why SST has changed are explored, such as the effects of climate change. 

## 3.2.2 Analysis

![](rolling_temp_average_rink_isbrae.png)

**Figure 7:** Average SST per year for the glacier Rink Isbrae, with a red dotted line fitted as the linear regression.

For the glacier Rink Isbrae, Figure 7 shows the nearest SST measurements. Looking at this plot, it is easy to see that there is an overall rise in SST over the past 50 years. From 1970 to 1980, the average temperature every year ranges between 0 – 0.5 °C and is shown to have a spike pattern. In 1982, the SST drops rapidly to lower than –0.5°C. It then rises rapidly from 1983 to 1988 to a temperature of 0.5°C. Overall, the SST fluctuates around an average of around 0°C. In 1996, the temperature rises significantly from around –0.25°C to around 1°C and slowly increases throughout the 2000s. In the 2010s temperatures are fluctuating around 1.25°C. 

This plot has a trendline fitted but this may not be right as this plot could also be interpreted as two level straight lines, one averaging around 0°C and another averaging at roughly 1.25°C. The trendline indicates a warming trend over the past 50 years, with approximate equation: SST = 0.0277 x Year +(-54.73). The gradient shows the amount of SST change within a year, so if the gradient is multiplied by 50, this gives a warming of 1.385°C over the time period. This can also be seen in Figure 7.

![](sst_area_regression_plot.png)

**Figure 8:** Relationship between SST and the Glacier Area for the glacier Rink Isbrae. the blue dots indicate each data recorded; the blue line is the linear regression line. The shading around the regression line shows the 95% confidence interval. The y axis is the Glacier Area with every value being multiplied by 30,000km, as shown at the top of the plot.

Figure 8 depicts a regression between SST and Glacier Area for Rink Isbrae, showing an inverse relationship also known as a negative correlation. It gives the evidence that SST does in fact affect the change in Glacier area and causes them to retreat. The 95% confidence interval is relatively small meaning the correlation is strong and uncertainty is low. The relationship between these two variables is strong as the data points tend to be clustered near the regression line.  It is possible that an increase in SST can be a main contributor to a retreat in glaciers due to melting.

![](sst_difference_map.png)

**Figure 9:** SST difference between the full year of 1970 and the full year of 2024. The largest increase in SST is shown by deep colours of red and the largest decrease in difference is shown in dark blues. The grey areas shown mostly on land and up north indicate no data or measurements made in this region.

Figure 9 clearly shows the difference in SST from the year 1970 to 2024. In this time there has been warming almost all-around Greenland, specifically further away from land. The darker red patches show the largest difference which is around 2 degrees in temperature in the last 50 years. 
 
There are some blue areas on the map showing an SST difference of around - 0.5 degrees Celsius to the south of Greenland. This could be due to the melting and retreat of glaciers with meltwater running into the sea meaning there is not as much change. Due to the retreat of these glaciers, the surface albedo is reduced, meaning there is a larger area of darker ocean which absorbs more heat and can lead to larger glacier ice loss.

## 3.2.3 Discussion

The large amount of greenhouse gases such as CO2 and methane means global air temperatures rise which in turn means that SST rises. Due to this, the Greenland glaciers continue to melt depositing large amounts of fresh water into the oceans. This can change the salinity and freshwater balance of the ocean and can change ocean circulation patterns. As air temperature, sea temperature and ocean circulation are linked, the climate in certain places will also change. 

Figure 8, the regression plot of Rink Isbrae, gives a great visual picture of how SST affects marine-terminating glaciers all around Greenland. It gives a good example of how warmer sea temperatures near glaciers have direct contact and can therefore increase glacier melting. 

The El Nino event in 1997 may have been a contributor to the drastic rise in temperature shown in Figure 7. According to Nasa, the El Nino event caused a major rise in Sea Surface Temperature, especially near the equator (Nagai, 2023). Due to ocean circulation, this warmer water will have increased salinity and may have been carried north, increasing ocean temperatures local to Greenland. 


![](ocean_circulation.png)

**Figure 10:** Ocean circulation patterns around Greenland (Understanding Global Change (n.d.)). 

The areas with the most SST difference follows the pattern of the labrador current in Figure 10. The Labrador current tends to bring cold water from up north along with cool air temperatures. The plot gives the evidence that the Labrador current is bringing warmer water from north. Another factor could be the salinity content changing due to the amount of fresh water from melting glaciers that is added. This shows signs that the Gulf Stream might be weakening. If this happens, it could potentially change ocean circulation patterns which in turn, will change air temperatures and cause environmental change.

![](annual_mean_temp_rink_isbrae.png)

**Figure 11:** Air temperature of RInk Isbrae over a time series of 50 years. the red line is the trendline showing a slight increase.

Looking at Figure 11, showing the air temperature over time of Rink Isbrae, the plot follows a very similar pattern to SST up until the early 2000s. There are three large temperature anomalies of cooling. A paper on Global and Planetary Change (Jiang, S., Ye, A. and Xiao, C. (2020)) has a similar plot and mentions that the larger decrease in temperature in the early 1980s and in the early 1990s may be due to volcanic activity. More specifically, the 1980 Mt St Helens Eruption, 1983 El chichón eruption and the 1991 Mt Pinatubo eruption (Volcano Hazards Program, 2015). Even though these are far away, volcanic eruptions affect the atmosphere globally due to gases like sulphur dioxide and aerosols. The Mt Pinatubo eruption released the one of the largest amounts of aerosols like ash and dust into the atmosphere, which reflected solar radiation back into space, cooling global air temperatures. Due to the cooling of air temperatures, SST also tend to drop due to their direct relationship.

# Section 4:
*(Authors: Ellie, 100%)*

## 4.1 Outline

We can look at a comparative regional analysis, identifying the largest retreat (Zachariae Isstrom in the northeast) and the largest advance (Qujuuttap Sermia in the west) from 1972 to 2022, considering localised sea surface and air temperature trends as drivers. Analysing these 2 cases and their contrasting behaviours can help us to identify if these factors have a considerable effect on area change across the whole of Greenland. 

## 4.2 Analysis

![](gain_loss_glacier_location_map.png)

**Figure 12:** Map of Greenland showing the locations of the glacier that have retreated the most (Zachariae Isstrom) and advanced the most (Qajuuttap Sermia). 

![](sst_1970_map.png)

**Figure 13a:** Sea surface temperature off the coast of Greenland in 1970

![](sst_2024_map.png)

**Figure 13b:** Sea surface temperature off the coasts of Greenland in 2024.

Figures 13a and 13b show a great representation of how sea surface temperatures have increased from 1970 until 2024. This means that for most of the marine-terminating glaciers in Greenland, they will have a similar result, glacier retreat. Looking at the location (Figure 12) of Zachariae Isstrom, the temperature in this area hasn’t changed a significant amount. Qajuuttap Sermia has approximately changed from 2°C to around 7°C.

![](zachariae_isstrom_series.png)

**Figure 14:** Timeseries of yearly average of the changes in the area (top) and corresponding air temperature (middle) and SST (bottoms) of Zachariae Isstrom Glacier from 1972 to 2022

![](qajuuttap_sermia_series.png)

**Figure 15:** Timeseries yearly average of the changes in the area (top) and corresponding air temperature (middle) and SST (bottoms) of Qajuuttap Sermia Glacier from 1972 to 2022

Zachariae Isstrom shows a stable area until the early 2000s showing only a slight, gradual decrease. This is followed by a marked and sustained retreat until 2012 where there was an abrupt and substantial drop falling from ~93,200 to ~92,600km2. After about 2013/2014 the area loss stabilises again suggesting rapid loss in 2012 was from a major structural collapse (iceberg calving) or a rapid retreat. 

Retreat rate coincides with an increased rise in air temperatures at the same time with an overall increase of ~4°C, although there is high variability. From ~2000 there is a more rapid warming compared to earlier years and a few episodes of steep increase, such as in ~2012 which again could be influenced by the Greenland heatwave as this also lines up with the rapid decrease episode explained in section 2.2, using Figure 3. This relationship highlights the strong influence of atmospheric warming on retreat in northeast Greenland.

The SST time series (shown in Figure 14) has temperatures ranging from ~-2 to 2°. The SST follows a different pattern to Qajuuttap Sermia as it is relatively stable and low, completely below 0°C from the mid-1970s until the early 2010s. There is an abrupt warming, again in 2012, where the localised SST experienced a sharp and dramatic increase jumping from just under -2°C to ~2°C in just a few years. Since this jump, SST has remained significantly warmer than before, though seems to take a decreasing pattern again from 2010 onwards. We can also see that this may not be following the same trend as other met stations data around Greenland in Figure 13, where the localised SST to the location of Zachariae Isstrom doesn’t decrease in temperature between 1970 and 2024 as much as the rest of the coasts.

The Sermia graph (Figure 15) shows an overall area increase of approximately 1.3$km^2$ over 54 years, whereas the Isstrom graph (Figure 14) shows an overall area decrease of 1050$km^2$. The advance is still only a fraction of the area of the glacier with the greatest retreat suggesting that, because such a small increase occurs over such a long period of time that it is like an anomaly with little influence on net ice balances. The graph does show a small decrease in area around 1979, holds this until around 1992 (accumulation and melt were likely equal) then gradually increases until around 2010. From 2010 Sermia grew significantly more in comparison to previous rates, indicating accumulation exceeded melt. 

The air temperature series it taken from the nearest met station to Sermia and although the data is highly variable there is a general trend toward higher temperatures where fluctuations are likely due to the change in seasons. The temperatures here are much cooler than those recorded at the nearest station to Zachariae Isstrom. Until the 2000s air temperatures frequently dropped below 0°C, especially between 1980-1995. After this temperature doesn’t reach lower than -2.5°C with significant spikes in temperature decrease around 2005, 2012 and 2016. 

From the plot, SST values range is ~2-5°C. Overall there is an increasing trend similar to the behaviour of air temperature, showing a long-term warming. Lower SSTs occurred in the early 1980s, and higher spikes of SSTs are more frequent from the 2000s onwards. 

Looking at these plots side by side suggests a lagged relationship between the 2 temperature factors and the change in area. The most significant increase in area (~2006/7) occurs during the same time as the clear warming trends in both air and sea surface temperatures start to increase slightly more rapidly. From this comparison we can see clearly that Qajuuttap Sermia exhibits unique behaviour, unlike the majority of the other glaciers, as area still increases despite warming. 

These time series for both Zachariae Isstrom and Qajuuttap Sermia are really helpful in giving the overall trend but are more difficult to interpret more deeply as other factors influencing glacier area change are not analysed alongside these. 

## 4.3 Discussion

Time series for Zachariae Isstrom and Qajuuttap Sermia are helpful in giving overall trend but are more difficult to interpret deeply as other factors influencing glacier area change are not analysed alongside these. 

Zachariae Isstrom is the glacier that has retreated most not only due to the rising air and sea surface temperatures but also the topography of the ocean floor at the head of the glacier (Mouginot et al., 2015). A positive feedback mechanism of glacier retreat is created as basal melting contributes to sea level rise and rising sea levels will continue to destabilize the marine portion of Zachariae Isstrom for decades (Mouginot et al., 2015). This leads to increased iceberg calving, causing levels to rise, causing more iceberg calving generating a cyclical pattern. Overall, this self-reinforcing cycle highlights how climate warming and ocean dynamics together accelerate the long-term destabilisation and retreat of Greenland glaciers, in particular Zachariae Isstrom. 

The Qajuuttap Sermia glacier, a tidal calving glacier, is a special case as shown in the above plots illustrating glacier area change, as it is actually the glacier that retreats the most albeit still only an almost negligible amount. Only 2 Greenland glaciers have experienced retreat – Heimdal Gletscher is the other one, with an even smaller advance. As suggested above, the reason for the slight increase in area of Sermia could be down to many different contributing factors such as infrequent ice calving, local mass balance conditions of ice cover in south Greenland, high precipitation rates on inland ice and therefore upland regions being susceptible to local strong variations resulting in periodic accumulation patterns (Weidick, 2009).  

This is where we consider the limitation of our data as we cannot distinguish why this unique behaviour occurs just from looking at air and SST as other factors, like topography, also contribute to the retreat or advance of glaciers. 


# Section 5: How might Greenland Glaciers continue to change?
*(Authors: Joel, 100%)*

## 5.1 Outline
Having analysed and formed hypothesis on why glaciers have changed around Greenland, a more pressing question is how these glaciers might continue to change. We will attempt to analyse this by looking at what trends have occurred, how we might extend these to the future, and how confident we are in these results.

## 5.2 Fitting Trends

Looking back to section 2.2, Figure 2 describes the percentage change of total glacier area since 1972. Upon examination, there appears to be three distinct, linear sections: 1972-1999, 2000-2012, and 2013-2023. We fit a linear regression to each of these section, shown in Figure 16 below.

![](percentage_change_linear_fits.png)

**Figure 16:** Timeseries of percentage change of total glacier area, with piecewise linear fits graphed over appropriate regions

Analysing these regressions, we see a sharp increase in the rate of change of area loss on the 1999/2000 boundary, and a sudden shift on the 2012/2013 boundary. Note that from 2012-2023 the rate of change is slightly less than the period before. Overall, on observation it is apparent that this piecewise approach will better approximate the underlying data than a regression over the whole set. Considering this, we will extend only our model from 2013- to 2030, to see how glacier area may continue to change.

## 5.3 Projections

![](pct_change_linear_extrapolation.png)

**Figure 17:** Extrapolation of linear fit 2013-2022 with bootstrapped confidence interval

Extending our time index to 2030, we continue and plot the model past 2022, with the vertical dashed line in Figure 17 denoting the start of extrapolation. To get a confidence interval, we bootstrap sample our data 1000 times, removing random data points and reevaluating the slope and intercept of the regression. The blue shaded area shows a 95% confidence interval of the regression. 

We can see that glaciers are expected to continue to retreat, to around -0.42%, a further 0.07% since 2022 (equivalent to approximately 920 square kilometres). In the probability density displayed in Figure 18 below, we see that the uncertainty calculated from the bootstrapping remains relatively small, with 95% confidence that the projected loss in 2030 will be between -0.4158% and -0.4247%, and a higher probability that the outcome will lie closer to our projected value. 

![](bootstrap_histogram.png)

**Figure 18:** Probability density histogram of projected percentage change of glacier area in 2030 from bootstrapped samples

Despite what may seem like an insignificant 0.07% further loss, in absolute terms this is around 920 square kilometres, an area larger than the land area of New York City (Lankevich, 2019). The implication of this is staggering, representing a substantial volume of ice transfer to the ocean within less than a decade. Such a loss not only contributes measurably to global sea-level rise but also signals the sustained imbalance of Greenland’s glacial systems. Continued retreat at this scale suggests that many outlet glaciers remain dynamically unstable, with events such as increased calving or surface melt likely to induce further mass loss. 

## 5.4 Limitations

It is worth noting that despite considering uncertainty, this model still has major limitations reducing its reliability. Firstly, we assume that the prediction period will follow the same trend as the observed period. Secondly, our prediction is based off only ten years of data. Finally, it is worth not that much glacier loss can be localised; our prediction accounts for overall glacier area change, but individual glaciers may have a much higher variability in results. Figure 19 shows some percentage of individual glaciers can be anywhere from negligible to 10%. It is also apparent that while some glaciers dominate absolute loss, their larger size means their percentage change is relatively low, another factor not accounted for in our model.

![](localised_losses_map.png)

**Figure 19:** Map of Greenland glaciers, size corresponding to absolute loss by 2022, and colour corresponding to percentage area loss by 2022, for each glacier

# Section 6: Conclusion
*(Authors: Russell, 40%; Ellie, 30%; Eliza, 30%)*

This report looks into glaciers on Greenland and how they have changed over the past 50 years by looking at glacial area, sea surface temperature and air temperature. The behaviours of these are analysed and explained with the help of research. We conclude that due to climate change, air temperature and sea surface temperatures are increasing which therefore means glacial areas are decreasing and glaciers are retreating. Air temperature has more of a correlated impact with the movement of glaciers but sea surface temperatures affect marine terminating glaciers like the ones investigated in this report. Both air temperature and sea surface temperatures increase basal melting and causing instability which then promotes iceberg calving. SST plots consistently shows a strong correlation with glacier area and evidence shows that climate change plays a large role in accelerating retreat. The change in glacier area is influenced by many factors with a couple being surface melting, ice calving and albedo effect. Not only does SST have an effect on melting of glaciers, but so does melting on SST. The freshwater input from melting glaciers can also alter ocean circulation. 
Overall, by looking back in the last 50 years, it is seen that the trend in glacier retreat is increasing and is likely to do so more in the future.


# Bibliography

Hall, S. (2022). NASA has weighed Greenland - and the speed of its ice loss. [online] World Economic Forum. Available at: https://www.weforum.org/stories/2022/10/greenland-melting-ice-climate-change-nasa/ 

NASA Earthdata. (2024). Glacier Power: What is Glacial Calving? | NASA Earthdata. [online] Available at: https://www.earthdata.nasa.gov/topics/cryosphere/glaciers/glacier-power/what-is-glacial-calving 

Datacamp.com. (2025). A function to do pairs bootstrap | Python. [online] Available at: https://campus.datacamp.com/courses/statistical-thinking-in-python-part-2/bootstrap-confidence-intervals?ex=12 [Accessed 10 Nov. 2025]. 

NOAA Climate.gov. (2012). Summer 2012 brought record-breaking melt to Greenland. [online] Available at: https://www.climate.gov/news-features/understanding-climate/summer-2012-brought-record-breaking-melt-greenland?utm_source=chatgpt.com  [Accessed 10 Nov. 2025].

Boyall, L. (2024). An introduction to the Greenland Ice Sheet. [online] AntarcticGlaciers.org. Available at: https://www.antarcticglaciers.org/glaciers-and-climate/changing-greenland-ice-sheet/greenland-ice-sheet/. 

King, M.D., Howat, I.M., Candela, S.G., Noh, M.J., Jeong, S., Noël, B.P.Y., van den Broeke, M.R., Wouters, B. and Negrete, A. (2020). Dynamic ice loss from the Greenland Ice Sheet driven by sustained glacier retreat. Communications Earth & Environment, [online] 1(1), pp.1–7. doi: https://doi.org/10.1038/s43247-020-0001-2.   

Zhao, F., et al. (2024) Linking glacier retreat with climate change on the Tibetan Plateau. The Cryosphere, 18, 5595-5614. https://tc.copernicus.org/articles/18/5595/2024/tc-18-5595-2024.pdf tc.copernicus.org

Williamson, S. N., Marshall, S. J., Menounos, B. (2025) Temperature mediated albedo decline portends acceleration of North American glacier mass loss. Communications Earth & Environment, 6, 555. https://doi.org/10.1038/s43247-025-02503-x 

Nagai, T. (2023). Gulf Stream Closes the Valve of the Labrador Current. [online] Eos. Available at: https://eos.org/editor-highlights/gulf-stream-closes-the-valve-of-the-labrador-current.

Jiang, S., Ye, A. and Xiao, C. (2020). The temperature increase in Greenland has accelerated in the past five years. Global and Planetary Change, 194, p.103297. doi: https://doi.org/10.1016/j.gloplacha.2020.103297

Volcano Hazards Program (2015). Volcanoes Can Affect Climate | U.S. Geological Survey. [online] www.usgs.gov. Available at: https://www.usgs.gov/programs/VHP/volcanoes-can-affect-climate.

Understanding Global Change (n.d.). Ocean circulation. [online] Understanding Global Change. Available at: https://ugc.berkeley.edu/background-content/ocean-circulation/

Mouginot, J., Rignot, E., Scheuchl, B., Fenty, I., Khazendar, A., Morlighem, M., Buzzi, A. and Paden, J. (2015). Fast retreat of Zachariae Isstrom, northeast Greenland. Science, 350(6266), pp.1357–1361. doi:https://doi.org/10.1126/science.aac7111  

Weidick, A. (2009). Johan Dahl Land, south Greenland: the end of a 20th century glacier expansion. Polar Record, 45(4), pp.337–350. doi:https://doi.org/10.1017/s003224740900833x.

Lankevich, G. (2019). New York City | Layout, People, Economy, Culture, & History. In: Encyclopædia Britannica. [online] Available at: https://www.britannica.com/place/New-York-City. 


# Appendix 1: Python Code
**A1.1 Data Import and Cleaning**

In [1]:
import os
import pandas as pd
import numpy as np
import matplotlib.pyplot as plt
import seaborn as sns
import netCDF4 as nc
from mpl_toolkits.basemap import Basemap
import matplotlib.dates as mdates
import cartopy.crs as ccrs
import cartopy.feature as cfeature

In [2]:
# set path to data directory
shared_dir_path = os.environ["SHARED_DATA_DIR"] + '/'
path = shared_dir_path + 'ESDA2_GroupProject_Data/'

In [3]:
# read glacier data
file = 'glacier_area.xlsx'
file_path = path + file
glacier_area = pd.read_excel(file_path)

In [4]:
# make a dict of each glacier location
glacier_locations = glacier_area[['Glacier', 'Latitude', 'Longitude']].set_index('Glacier').T
glacier_locations = glacier_locations.rename(index={'Latitude': 'lat', 'Longitude': 'lon'})
glacier_locations = glacier_locations.to_dict()

In [5]:
# drop lat/lon data and convert to timeseries index
glacier_timeseries = glacier_area.drop(['Latitude', 'Longitude'], axis=1).set_index('Glacier').T
glacier_timeseries.index = pd.to_datetime(glacier_timeseries.index)

In [6]:
# read station metdata
file = '420800.csv'
file_path = path + file
station_420800 = pd.read_csv(file_path, delimiter=';')

file = '422100.csv'
file_path = path + file
station_422100 = pd.read_csv(file_path, delimiter=';')

file = '425000.csv'
file_path = path + file
station_425000 = pd.read_csv(file_path, delimiter=';')

file = '430100.csv'
file_path = path + file
station_430100 = pd.read_csv(file_path, delimiter=';')

file = '432000.csv'
file_path = path + file
station_432000 = pd.read_csv(file_path, delimiter=';')

file = '436000.csv'
file_path = path + file
station_436000 = pd.read_csv(file_path, delimiter=';')

file = 'station_details.xlsx'
file_path = path + file
station_details = pd.read_excel(file_path)

In [7]:
# clean station data to be airtemp timeseries
metdata_420800 = station_420800.rename(columns={'Hour(utc)': 'Hour', '101':'airtemp'})
metdata_420800['Date'] = pd.to_datetime(metdata_420800[['Year', 'Month', 'Day', 'Hour']])
metdata_420800 = metdata_420800.set_index('Date')
metdata_420800 = metdata_420800[['airtemp']]

metdata_422100 = station_422100.rename(columns={'Hour(utc)': 'Hour', '101':'airtemp'})
metdata_422100['Date'] = pd.to_datetime(metdata_422100[['Year', 'Month', 'Day', 'Hour']])
metdata_422100 = metdata_422100.set_index('Date')
metdata_422100 = metdata_422100[['airtemp']]

metdata_425000 = station_425000.rename(columns={'Hour(utc)': 'Hour', '101':'airtemp'})
metdata_425000['Date'] = pd.to_datetime(metdata_425000[['Year', 'Month', 'Day', 'Hour']])
metdata_425000 = metdata_425000.set_index('Date')
metdata_425000 = metdata_425000[['airtemp']]

metdata_430100 = station_430100.rename(columns={'Hour(utc)': 'Hour', '101':'airtemp'})
metdata_430100['Date'] = pd.to_datetime(metdata_430100[['Year', 'Month', 'Day', 'Hour']])
metdata_430100 = metdata_430100.set_index('Date')
metdata_430100 = metdata_430100[['airtemp']]

metdata_432000 = station_432000.rename(columns={'Hour(utc)': 'Hour', '101':'airtemp'})
metdata_432000['Date'] = pd.to_datetime(metdata_432000[['Year', 'Month', 'Day', 'Hour']])
metdata_432000 = metdata_432000.set_index('Date')
metdata_432000 = metdata_432000[['airtemp']]

metdata_436000 = station_436000.rename(columns={'Hour(utc)': 'Hour', '101':'airtemp'})
metdata_436000['Date'] = pd.to_datetime(metdata_436000[['Year', 'Month', 'Day', 'Hour']])
metdata_436000 = metdata_436000.set_index('Date')
metdata_436000 = metdata_436000[['airtemp']]

# set a dict to make each station accessible later
metdata = {
    '420800': metdata_420800,
    '422100': metdata_422100,
    '425000': metdata_425000,
    '430100': metdata_430100,
    '432000': metdata_432000,
    '436000': metdata_436000
}

In [8]:
# read sst netCDF file
file = 'HadISST_SST_northatlantic.nc'
file_path = path + file
ds = nc.Dataset(file_path)

# isolate variables
sst_timedelta = ds.variables['time'][:]
sst_lat = ds.variables['latitude'][:]
sst_lon = ds.variables['longitude'][:]
sst = ds.variables['sst'][:,:,:]

In [9]:
# convert time from start to a date
sst_time = pd.to_timedelta(sst_timedelta, unit='D') + pd.Timestamp('1870-01-01 00:00:00')

In [10]:
# map below freezing values to frozen value (1.8)
mask_value = -1.8
freezing_point = -1.8
sst[sst < mask_value] = freezing_point

**A1.2 Functions**

In [11]:
# pick what SST values corresponds to chosen glacier
def pick_closest_SST(lats, lons, sst, glacier):
    """
    lats: 1D array of latitudes
    lons: 1D array of longitudes
    sst: 3D array of sea surface temp at (time, latitude, longitude)
    glacier: (lat, lon); a tuple of glacier coordinates

    returns: ind_lat, ind_lon -> indices to get sst data closest to glacier
    """
    sst_slice = sst[-1, :, :]   # last time step
    mask = sst_slice.mask
    
    lon2d, lat2d = np.meshgrid(lons, lats)
    
    dist = np.sqrt((lat2d - glacier[0])**2 + (lon2d - glacier[1])**2)

    dist_masked = np.ma.array(dist, mask=mask)

    pair = np.unravel_index(np.argmin(dist_masked), dist_masked.shape)

    ind_lat = pair[0]
    ind_lon = pair[1]
    
    return ind_lat, ind_lon

In [12]:
# find closest station (airtemp) for chosen glacier
def pick_closest_airtemp(station_details, glacier_location):
    """
    station_details: dataframe with columns 'Station Code', 'Latitude', 'Longitude'
    target_lat: latitude of glacier
    target_lon: longitude of glacier

    returns: station_code -> code of closest station
    """
    target_lat, target_lon = glacier_location
    lats = station_details['Latitude'].values
    lons = station_details['Longitude'].values
    station_codes = station_details['Station number'].values

    dist = np.sqrt((lats - target_lat)**2 + (lons - target_lon)**2)

    min_index = np.argmin(dist)

    station_number = station_codes[min_index]

    return station_number   

In [13]:
# create df of robust yearly averaged values (inner join) for chosen glacier
def get_glacier_df(GLACIER, SAMPLE_TIME):

    target_location = (glacier_locations[GLACIER]['lat'], glacier_locations[GLACIER]['lon'])

    # get data from closest met station
    station_number = pick_closest_airtemp(station_details, target_location)
    glacier_metdata = metdata[str(station_number)]

    # take only year where we have data for all months - avoids bias from missing months, as temp varies seasonally
    glacier_metdata = glacier_metdata[~glacier_metdata['airtemp'].isna()]
    months_per_year = glacier_metdata.groupby(glacier_metdata.index.year).apply(lambda x: x.index.month.nunique())
    valid_years = months_per_year[months_per_year == 12].index
    glacier_metdata = glacier_metdata[glacier_metdata.index.year.isin(valid_years)]

    # get SST data from closest grid point and convert to time series
    ind_lat, ind_lon = pick_closest_SST(sst_lat, sst_lon, sst, target_location)
    glacier_sst = sst[:, ind_lat, ind_lon]
    glacier_sst = pd.DataFrame({'Date': sst_time, 'SST': glacier_sst})
    glacier_sst = glacier_sst.set_index('Date')

    # align time series to common start date
    start = max(glacier_timeseries.index[0], glacier_metdata.index[0], glacier_sst.index[0])

    glacier_area_series = glacier_timeseries[GLACIER][glacier_timeseries.index >= start]
    glacier_metdata_series = glacier_metdata[glacier_metdata.index >= start]
    glacier_sst_series = glacier_sst[glacier_sst.index >= start]

    # resample time series
    glacier_area_resampled = glacier_area_series.resample(SAMPLE_TIME).mean().dropna()
    glacier_metdata_resampled = glacier_metdata_series.resample(SAMPLE_TIME).mean().dropna()
    glacier_sst_resampled = glacier_sst_series.resample(SAMPLE_TIME).mean().dropna()

    # join time series into single dataframe
    glacier_df = pd.concat([glacier_area_resampled, glacier_metdata_resampled, glacier_sst_resampled], axis=1, join='inner')
    glacier_df = glacier_df.rename(columns={GLACIER: 'area'})
    
    return glacier_df

In [14]:
# bootstrap function for linear regression (Datacamp, 2025)
def draw_bs_pairs_linreg(x, y, size=1):
    """Perform pairs bootstrap for linear regression."""

    # Set up array of indices to sample from: inds
    inds = np.arange(0, len(x))

    # Initialize replicates: bs_slope_reps, bs_intercept_reps
    bs_slope_reps = np.empty(size)
    bs_intercept_reps = np.empty(size)

    # Generate replicates
    for i in range(size):
        bs_inds = np.random.choice(inds, size=len(inds)) # selected with replacement - omits random data points per sample
        bs_x, bs_y = x[bs_inds], y[bs_inds]
        bs_slope_reps[i], bs_intercept_reps[i] = np.polyfit(bs_x, bs_y, 1)

    return bs_slope_reps, bs_intercept_reps

Above code is taken from Datacamp, Statistical Thinking in Python Part 2 (Datacamp, 2025).

**A1.3 Report Section 2**

In [15]:
# FIGURE 1
# create and plot map of glacier locations
m = Basemap(
    projection='lcc',  # Lambert Conformal Conic
    lat_0=70,          # Center latitude
    lon_0=-45,         # Center longitude
    lat_1=60,          # First standard parallel
    lat_2=80,          # Second standard parallel
    llcrnrlat=58,      # Lower left corner latitude
    urcrnrlat=75,      # Upper right corner latitude
    llcrnrlon=-60,     # Lower left corner longitude
    urcrnrlon=25,     # Upper right corner longitude
    resolution='i',    # Intermediate resolution
    area_thresh=1000   # Minimum area for coastlines
)

# Draw map features
m.drawcoastlines(linewidth=0.5)
m.drawcountries(linewidth=1)
m.fillcontinents(color='lightgray', lake_color='lightblue')
m.drawmapboundary(fill_color='lightblue')

# Add gridlines
m.drawmeridians(range(-80, 0, 10), labels=[0,0,0,1], fontsize=8)
m.drawparallels(range(60, 85, 5), labels=[1,0,0,0], fontsize=8)

# get xy coordinated of glaciers from lat/lon
lons = [glacier_locations[key]['lon'] for key in glacier_locations]
lats = [glacier_locations[key]['lat'] for key in glacier_locations]
x, y = m(lons, lats)

# Plot the point locaitons
m.plot(x, y, 'ro', markersize=2, label='Glacier')

# customise plot
plt.title('Location of Greenland Glaciers', fontsize=14)
plt.legend(bbox_to_anchor=(1,1), loc='upper right')
plt.savefig('greenland_glacier_location')
plt.close()

In [16]:
# FIGURE 2 
# timersiers plot of total area % change in glacier area
# calculate percentage loss
sum_areas = glacier_timeseries.sum(axis=1)
pct_loss = (sum_areas - sum_areas.iloc[0]) / sum_areas.iloc[0] * 100 # percentage

# plot percentage loss
plt.figure(figsize=(10,6))
plt.plot(pct_loss.index, pct_loss)
plt.xlabel('Date')
plt.ylabel('Area Change since 1972 (%)')
plt.title('Percentage Total Glacier Area Change, 1972-2022')
plt.grid()
plt.savefig('percentage_total_area_change')
plt.close()

In [17]:
# what does 0.1% correlate to
print(f"{sum_areas.iloc[0]*0.1/100:.2f}km^2")

1317.42km^2


In [18]:
# how much total area lost 1972-2022
print(f"{sum_areas.iloc[0] - sum_areas.iloc[-1]:.2f}km^2")

4725.17km^2


In [19]:
# FIGURE 3
# area change and change in mean of all glaciers
# calculate change in area
delta_df = glacier_timeseries - glacier_timeseries.iloc[0]

# set figure and axis
fig, ax = plt.subplots(1,1, figsize=(10,6))

# plot timeseires for all glaciers as 'background'
for glacier in delta_df:
    delta_df[glacier].plot(ax=ax, color='grey', alpha=0.2, label='_nolegend_')

#plot mean and mean in 2022 
delta_df.mean(axis=1).plot(ax=ax, color='red', label='Mean')
mean_2022 = delta_df.mean(axis=1).iloc[-1]
plt.axhline(mean_2022, linestyle='--', color='firebrick', alpha=0.5, label=f'Mean in 2022 ({mean_2022:.2f})')

# customise plot
plt.xlabel('Date')
plt.ylabel(r'Area Change ($km^2$)')
plt.title('Individual Glacier Area Change, 1972-2022')
plt.legend()
plt.grid()
plt.savefig('individual_glacier_area_change')
plt.close()

**A1.4 Report Section 3.1** 

In [20]:
# FIGURE 4
# annual mean airtemp close to glacier of choice
# choose glacier
GLACIER = 'Zachariae Isstrom'

# find closest station
target_location = (glacier_locations[GLACIER]['lat'], glacier_locations[GLACIER]['lon'])
station_number = pick_closest_airtemp(station_details, target_location)
airtemp_data = metdata[str(station_number)]

# get robust yearly averages (full yearly data)
airtemp_data_r = airtemp_data[~airtemp_data['airtemp'].isna()]
mpy = airtemp_data_r.groupby(airtemp_data_r.index.year).apply(lambda x: x.index.month.nunique())
vy = mpy[mpy == 12].index
airtemp_data_r = airtemp_data_r[airtemp_data_r.index.year.isin(vy)]

yearly_airtemp = airtemp_data_r.resample('YE').mean().dropna()


# plot and customise graph
plt.figure(figsize=(10,5))
plt.plot(yearly_airtemp, color='firebrick')
plt.title(f'Annual Mean Air Temperautre near {GLACIER} (station {station_number})')
plt.ylabel('Temperature (°C)')
plt.xlabel('Date')
plt.grid()
plt.savefig('annual_mean_temp_432000')
plt.close()

In [21]:
# FIGURE 5
# plot a pairwise yearly mean scatter plot of area vs airtemp
GLACIER = 'Rink Isbrae'
SAMPLE_TIME = 'YE'
X = 'airtemp'
Y = 'area'

# set labels dict and get concurrent df
labels = {'airtemp': 'Air Temperature (°C)',
    'SST': 'Sea Surface Temperature (°C)',
    'area': r'Glacier Area ($km^2$)'}
glacier_df = get_glacier_df(GLACIER, SAMPLE_TIME)

# make regression plot with 95% confidence interval
sns.regplot(x=X, y=Y, data=glacier_df, ci=95, line_kws={'label': 'Linear Regression with 95% CI'})
plt.xlabel(labels[X])
plt.ylabel(labels[Y])
plt.title(f'{labels[X]} vs {labels[Y]} for {GLACIER}')
plt.legend()
plt.savefig('airtemp_area_regression')
plt.close()

In [22]:
# FIGURE 6
# plot Zachariae Isstrom area and airtemp concurrently
GLACIER = 'Zachariae Isstrom'

# get concurrent df
df = get_glacier_df(GLACIER, 'YE')

# plot and customise graph
fig, ax = plt.subplots(2, 1, figsize=(10,10))
ax[0].set_title(f'{GLACIER} Area concurrent with Annual Mean Air Temperatures, 1972-2022')
ax[0].plot(df['area'])
ax[0].set_xticklabels([])
ax[0].set_ylabel(r'Glacier Area $km^2$')
ax[0].grid()
ax[1].plot(df['airtemp'], color='firebrick')
ax[1].set_ylabel('Air Temperature (°C)')
ax[1].set_xlabel('Year', size=12)
ax[1].grid()

plt.savefig('zachariae_isstrom_concurrent_plot')
plt.close()

**A1.5 Report Section 3.2**

In [23]:
# FIGURE 7
# rolling yearly average sst with linear regression for rink isbrae 
GLACIER = 'Rink Isbrae'
START_TIME = '1970-01-01'
END_TIME = '2024-12-12'

# get closest sst to location
target_location = (glacier_locations[GLACIER]['lat'], glacier_locations[GLACIER]['lon'])
ind_lat, ind_lon = pick_closest_SST(sst_lat, sst_lon, sst, target_location)
glacier_sst = sst[:, ind_lat, ind_lon]

# restrict timeframe
restricted_time = (sst_time >= START_TIME) & (sst_time <= END_TIME)
sst_time_plot = sst_time[restricted_time]
glacier_sst_plot = glacier_sst[restricted_time]

# set up df for plotting
plot_df = pd.DataFrame({'sst': glacier_sst_plot, 'date': sst_time_plot})
plot_df = plot_df.set_index('date')

# find trend of rolling average
rolling_mean = plot_df.rolling(window=12).mean()
valid_data = rolling_mean.dropna()
m, b = np.polyfit(mdates.date2num(valid_data.index), valid_data.values, 1)

# plot and customise
plt.figure(figsize=(10,5))
plt.plot(plot_df.rolling(window=12).mean())
plt.plot(valid_data.index, m*mdates.date2num(valid_data.index) + b, color='red', linestyle='--', label='Linear Regression', linewidth=2)
plt.title(f'Rolling Yearly Average Sea Surface Temperature; {GLACIER}')
plt.xlabel('Time')
plt.ylabel('Sea Surface Temperature (°C)')
plt.grid()
plt.legend()
plt.savefig('rolling_temp_average_rink_isbrae')
plt.close()

In [24]:
# FIGURE 8
# plot a pairwise yearly mean scatter plot of area vs sst
GLACIER = 'Rink Isbrae'
SAMPLE_TIME = 'YE'
X = 'SST'
Y = 'area'

# set labels dict and get concurrent df
labels = {'airtemp': 'Air Temperature (°C)',
    'SST': 'Sea Surface Temperature (°C)',
    'area': r'Glacier Area ($km^2$)'}
glacier_df = get_glacier_df(GLACIER, SAMPLE_TIME)

# make regression plot with 95% confidence interval
sns.regplot(x=X, y=Y, data=glacier_df, ci=95, line_kws={'label': 'Linear Regression with 95% CI'})
plt.xlabel(labels[X])
plt.ylabel(labels[Y])
plt.title(f'{labels[X]} vs {labels[Y]} for {GLACIER}')
plt.legend()
plt.savefig('sst_area_regression_plot')
plt.close()

In [25]:
# FIGURE 9
ind_70s = np.where((sst_time > '1970-01-01') & (sst_time < '1970-12-31'))[0]
sst_70s_mean = np.mean(sst[ind_70s, :, :], axis=0) 

ind_20s = np.where((sst_time > '2024-01-01') & (sst_time < '2024-12-31'))[0]
sst_20s_mean = np.mean(sst[ind_20s, :, :], axis=0)

sst_diff = sst_20s_mean - sst_70s_mean 

lon_grid, lat_grid = np.meshgrid(sst_lon, sst_lat)

plt.figure(figsize=(10, 8))
ax = plt.axes(projection=ccrs.PlateCarree())
ax.add_feature(cfeature.COASTLINE, linewidth=0.8)
ax.add_feature(cfeature.BORDERS, linewidth=0.5, linestyle='--')
ax.add_feature(cfeature.LAND, color='lightgray')
ax.set_extent([ sst_lon.min(), sst_lon.max(), sst_lat.min(), sst_lat.max() ]) 
mesh = ax.pcolormesh(lon_grid, lat_grid, sst_diff, cmap='coolwarm', vmin=-2, vmax=2, transform=ccrs.PlateCarree() ) 
cbar = plt.colorbar(mesh, label='SST Difference (°C)', shrink=0.7, orientation='horizontal')
ax.set_title('SST Difference: 1970 - 2024', fontsize=12) 
ax.set_xticks(np.arange(sst_lon.min(), sst_lon.max()+1, 10), crs=ccrs.PlateCarree())
ax.set_xticks(np.arange(sst_lon.min(), sst_lon.max()+1, 10), crs=ccrs.PlateCarree())
ax.set_yticks(np.arange(sst_lat.min(), sst_lat.max()+1, 10), crs=ccrs.PlateCarree()) 
ax.set_xlabel('Longitude')
ax.set_ylabel('Latitude') 
plt.tight_layout()
plt.savefig('sst_difference_map')
plt.close()

In [26]:
# FIGURE 11
# plot of airtemperature near rink isbrae over same time period with trend line
GLACIER = 'Rink Isbrae'

# find closest station
target_location = (glacier_locations[GLACIER]['lat'], glacier_locations[GLACIER]['lon'])
station_number = pick_closest_airtemp(station_details, target_location)
airtemp_data = metdata[str(station_number)]

# filter to be after 1970
airtemp_data = airtemp_data[airtemp_data.index >= '1970-01-01']

# get robust yearly averages (full yearly data)
airtemp_data_r = airtemp_data[~airtemp_data['airtemp'].isna()]
mpy = airtemp_data_r.groupby(airtemp_data_r.index.year).apply(lambda x: x.index.month.nunique())
vy = mpy[mpy == 12].index
airtemp_data_r = airtemp_data_r[airtemp_data_r.index.year.isin(vy)]

yearly_airtemp = airtemp_data_r.resample('YE').mean().dropna()

# fit linear regression
m, b = np.polyfit(mdates.date2num(yearly_airtemp.index), yearly_airtemp.values, 1)


# plot and customise graph
plt.figure(figsize=(10,5))
plt.plot(yearly_airtemp)
plt.plot(yearly_airtemp.index, m*mdates.date2num(yearly_airtemp.index) + b, color='red', linestyle='--', label='Linear Regression', linewidth=2)
plt.title(f'Annual Mean Air Temperautre; {GLACIER}')
plt.ylabel('Temperature (°C)')
plt.xlabel('Time')
plt.grid()
plt.legend()
plt.savefig('annual_mean_temp_rink_isbrae')
plt.close()

**A1.6 Report Section 4**

In [27]:
# FIGURE 12
# set up map projection
m = Basemap(
    projection='lcc',  # Lambert Conformal Conic
    lat_0=70,          # Center latitude
    lon_0=-45,         # Center longitude
    lat_1=60,          # First standard parallel
    lat_2=80,          # Second standard parallel
    llcrnrlat=58,      # Lower left corner latitude
    urcrnrlat=75,      # Upper right corner latitude
    llcrnrlon=-60,     # Lower left corner longitude
    urcrnrlon=25,     # Upper right corner longitude
    resolution='i',    # Intermediate resolution
    area_thresh=1000   # Minimum area for coastlines
)

# Draw map features
m.drawcoastlines(linewidth=0.5)
m.drawcountries(linewidth=1)
m.fillcontinents(color='lightgray', lake_color='lightblue')
m.drawmapboundary(fill_color='lightblue')

# Add gridlines
m.drawmeridians(range(-80, 0, 10), labels=[0,0,0,1], fontsize=8)
m.drawparallels(range(60, 85, 5), labels=[1,0,0,0], fontsize=8)

gain_glacier = 'Qajuuttap Sermia'
gain_lat = glacier_locations[gain_glacier]['lat']
gain_lon = glacier_locations[gain_glacier]['lon']
x_g, y_g = m(gain_lon, gain_lat)

loss_glacier = 'Zachariae Isstrom'
loss_lat = glacier_locations[loss_glacier]['lat']
loss_lon = glacier_locations[loss_glacier]['lon']
x_l, y_l = m(loss_lon, loss_lat)

plt.plot(x_g, y_g, marker='o', linestyle='none', color='blue', markersize=5, label=gain_glacier)
plt.plot(x_l, y_l, marker='o', linestyle='none',  color='red', markersize=5, label=loss_glacier)
plt.title('Glacier Locations')
plt.legend(bbox_to_anchor=(0.43,0.18), loc='upper left')
plt.savefig('gain_loss_glacier_location_map')
plt.close()

In [28]:
# FIGURE 13a
# map of Greenland SST, 1970
# set up map projection
m = Basemap(
    projection='lcc',  # Lambert Conformal Conic
    lat_0=70,          # Center latitude
    lon_0=-45,         # Center longitude
    lat_1=60,          # First standard parallel
    lat_2=80,          # Second standard parallel
    llcrnrlat=58,      # Lower left corner latitude
    urcrnrlat=75,      # Upper right corner latitude
    llcrnrlon=-60,     # Lower left corner longitude
    urcrnrlon=25,     # Upper right corner longitude
    resolution='i',    # Intermediate resolution
    area_thresh=1000   # Minimum area for coastlines
)

# Draw map features
m.drawcoastlines(linewidth=0.5)
m.drawcountries(linewidth=1)
m.fillcontinents(color='lightgray', lake_color='lightblue')
m.drawmapboundary(fill_color='lightblue')

# Add gridlines
m.drawmeridians(range(-80, 0, 10), labels=[0,0,0,1], fontsize=8)
m.drawparallels(range(60, 85, 5), labels=[1,0,0,0], fontsize=8)

# get x and y from lat lon grid data
lon_grid, lat_grid = np.meshgrid(sst_lon, sst_lat)
x, y = m(lon_grid, lat_grid)

# find idx of 1970
date_idx = np.where(sst_time >= '1970-01-01')[0][0]

# get max and min for concurrent scales
vmin = min(sst[-1, :, :].min(), sst[date_idx, :, :].min())
vmax = max(sst[-1, :, :].max(), sst[date_idx, :, :].max())

# plot
plt.pcolormesh(x, y, sst[date_idx, :, :], cmap='coolwarm', vmin=vmin, vmax=vmax)
plt.colorbar(label='Sea Surface Temperature (°C)', shrink=0.5, pad=0.1)
plt.title('Greenland Sea Surface Temperature, 1970 (°C)')
plt.savefig('sst_1970_map')
plt.close()

In [29]:
# FIGURE 13b
# map ofGreenland SST, 2024
# set up map projection
m = Basemap(
    projection='lcc',  # Lambert Conformal Conic
    lat_0=70,          # Center latitude
    lon_0=-45,         # Center longitude
    lat_1=60,          # First standard parallel
    lat_2=80,          # Second standard parallel
    llcrnrlat=58,      # Lower left corner latitude
    urcrnrlat=75,      # Upper right corner latitude
    llcrnrlon=-60,     # Lower left corner longitude
    urcrnrlon=25,     # Upper right corner longitude
    resolution='i',    # Intermediate resolution
    area_thresh=1000   # Minimum area for coastlines
)

# Draw map features
m.drawcoastlines(linewidth=0.5)
m.drawcountries(linewidth=1)
m.fillcontinents(color='lightgray', lake_color='lightblue')
m.drawmapboundary(fill_color='lightblue')

# Add gridlines
m.drawmeridians(range(-80, 0, 10), labels=[0,0,0,1], fontsize=8)
m.drawparallels(range(60, 85, 5), labels=[1,0,0,0], fontsize=8)

# get x and y grid from lon and lat
lon_grid, lat_grid = np.meshgrid(sst_lon, sst_lat)
x, y = m(lon_grid, lat_grid)

# plot
plt.pcolormesh(x, y, sst[-1, :, :], cmap='coolwarm', vmin=vmin, vmax=vmax)
plt.colorbar(label='Sea Surface Temperature (°C)', shrink=0.5, pad=0.1)
plt.title('Greenland Sea Surface Temperature, 2024 (°C)')
plt.savefig('sst_2024_map')
plt.close()

In [30]:
# FIGURE 14
GLACIER = 'Zachariae Isstrom'
SAMPLE_TIME = 'YE' # Year: 'YE', 5-Year: '5YE'

glacier_df = get_glacier_df(GLACIER, SAMPLE_TIME)

fig, ax = plt.subplots(3, 1, figsize=(10, 5))
ax[0].plot(glacier_df.index, glacier_df['area'])
ax[0].set_title(f'Glacier Area Time Series of {GLACIER}')
ax[1].plot(glacier_df.index, glacier_df['airtemp'], color='orange')
ax[1].set_title(f'Air Temperature Time Series of {GLACIER}')
ax[2].plot(glacier_df.index, glacier_df['SST'], color='green')
ax[2].set_title(f'Sea Surface Temperature Time Series at {GLACIER}')
plt.tight_layout()
plt.savefig('zachariae_isstrom_series')
plt.close()

In [31]:
# FIGURE 15
GLACIER = 'Qajuuttap Sermia'
SAMPLE_TIME = 'YE' # Year: 'YE', 5-Year: '5YE'

glacier_df = get_glacier_df(GLACIER, SAMPLE_TIME)

fig, ax = plt.subplots(3, 1, figsize=(10, 5))
ax[0].plot(glacier_df.index, glacier_df['area'])
ax[0].set_title(f'Glacier Area Time Series of {GLACIER}')
ax[1].plot(glacier_df.index, glacier_df['airtemp'], color='orange')
ax[1].set_title(f'Air Temperature Time Series of {GLACIER}')
ax[2].plot(glacier_df.index, glacier_df['SST'], color='green')
ax[2].set_title(f'Sea Surface Temperature Time Series at {GLACIER}')
plt.tight_layout()
plt.savefig('qajuuttap_sermia_series')
plt.close()

**A1.7 Report Section 5**

In [32]:
# FIGURE 16
# split into 3 distinct sections based on visual inspection
pct_loss_1972_1999 = pct_loss[(pct_loss.index < '2000-01-01')]
pct_loss_2000_2012 = pct_loss[(pct_loss.index >= '2000-01-01') & (pct_loss.index < '2013-01-01')]
pct_loss_2013_2023 = pct_loss[pct_loss.index >= '2013-01-01']

# fit linear to each section
m1, b1 = np.polyfit(mdates.date2num(pct_loss_1972_1999.index), pct_loss_1972_1999.values, 1)
m2, b2 = np.polyfit(mdates.date2num(pct_loss_2000_2012.index), pct_loss_2000_2012.values, 1)
m3, b3 = np.polyfit(mdates.date2num(pct_loss_2013_2023.index), pct_loss_2013_2023.values, 1)

# plot each section with its fit
plt.figure(figsize=(10,6))
plt.plot(pct_loss.index, pct_loss, label='Observed Data', color='grey')
plt.plot(pct_loss_1972_1999.index, m1*mdates.date2num(pct_loss_1972_1999.index) + b1, color='blue', label='Fit 1972-1999')
plt.plot(pct_loss_2000_2012.index, m2*mdates.date2num(pct_loss_2000_2012.index) + b2, color='green', label='Fit 2000-2012')
plt.plot(pct_loss_2013_2023.index, m3*mdates.date2num(pct_loss_2013_2023.index) + b3, color='red', label='Fit 2013-2023')
plt.xlabel('Date')
plt.ylabel('Area Change since 1972 (%)')
plt.title('Percentage Total Area Change since 1972 with Piecewise Linear Fits')
plt.legend()
plt.grid()
plt.savefig('percentage_change_linear_fits')
plt.close()

In [33]:
# FIGURE 17
# 2013 onwards for future projections with bootstrap confidence intervals

# set future dates
future_dates = pd.date_range(start=pct_loss_2013_2023.index[-1], end='2030-01-01', freq='ME')
plot_dates = pct_loss_2013_2023.index.append(future_dates)

# create linear model
m, b = np.polyfit(mdates.date2num(pct_loss_2013_2023.index), pct_loss_2013_2023.values, 1)
prediction = m*mdates.date2num(plot_dates) + b

# bootstrap for model uncertainty
# draw 1000 bootstrap replicates
x = mdates.date2num(pct_loss_2013_2023.index)
y = pct_loss_2013_2023.values
bs_slope_reps, bs_intercept_reps = draw_bs_pairs_linreg(x, y, size=1000)

# predict and get confidence intervals
bs_predictions = np.array([bs_slope_reps[i]*mdates.date2num(plot_dates) + bs_intercept_reps[i] for i in range(1000)])
ci_95 = np.percentile(bs_predictions, [2.5, 97.5], axis=0)


# plot bootstrap lines
plt.figure(figsize=(10,6))
plt.plot(pct_loss_2013_2023.index, pct_loss_2013_2023.values, label='Observed Data', color='grey')
plt.fill_between(plot_dates, ci_95[0], ci_95[1], color='blue', alpha=0.3, label='95% CI')
plt.axvline(pct_loss_2013_2023.index[-1], color='black', alpha=0.2, linestyle='--')
plt.plot(plot_dates, prediction, color='red', label='Prediction')
plt.xlabel('Date')
plt.ylabel('Area Change since 1972 (%)')
plt.title('Percentage Total Area Change with Bootstrapped Projections from 2013 Onwards')
plt.grid()
plt.legend()
plt.savefig('pct_change_linear_extrapolation')
plt.close()

In [34]:
# stats by 2030
pred_2030 = prediction[-1]
ci_2030 = ci_95[:, -1]
print(f"Area Change in {pct_loss_2013_2023.index[-1].year}: {pct_loss_2013_2023.values[-1]:.4f}%")
print(f"Projected area Change by 2030: {pred_2030:.4f}%")
print(f"95% CI for 2030: [{ci_2030[0]:.4f}%, {ci_2030[1]:.4f}%]")

Area Change in 2022: -0.3587%
Projected area Change by 2030: -0.4203%
95% CI for 2030: [-0.4251%, -0.4157%]


In [35]:
#FIGURE 18
# histogram of area loss in 2030 from bootstrap projections with 95% CI
plt.figure(figsize=(10,6))
plt.hist(bs_predictions[:, -1], bins=30, density=True, alpha=0.7, color='blue')
plt.xlabel('Projected Area Loss in 2030 (%)')
plt.ylabel('Density')
plt.title('Histogram of Projected Area Loss in 2030 from Bootstrap Projections')
plt.axvline(ci_95[0, -1], color='red', linestyle='dashed', linewidth=1, label='95% CI')
plt.axvline(ci_95[1, -1], color='red', linestyle='dashed', linewidth=1)
plt.axvline(prediction[-1], color='Orange', linestyle='solid', linewidth=1, label='Predicted Value')
plt.grid()
plt.legend()
plt.savefig('bootstrap_histogram')
plt.close()

In [36]:
# FIGURE 19
# set up map projection
m = Basemap(
    projection='lcc',  # Lambert Conformal Conic
    lat_0=70,          # Center latitude
    lon_0=-45,         # Center longitude
    lat_1=60,          # First standard parallel
    lat_2=80,          # Second standard parallel
    llcrnrlat=58,      # Lower left corner latitude
    urcrnrlat=75,      # Upper right corner latitude
    llcrnrlon=-60,     # Lower left corner longitude
    urcrnrlon=25,     # Upper right corner longitude
    resolution='i',    # Intermediate resolution
    area_thresh=1000   # Minimum area for coastlines
)

# Draw map features
m.drawcoastlines(linewidth=0.5)
m.drawcountries(linewidth=1)
m.fillcontinents(color='lightgray', lake_color='lightblue')
m.drawmapboundary(fill_color='lightblue')

# Add gridlines
m.drawmeridians(range(-80, 0, 10), labels=[0,0,0,1], fontsize=8)
m.drawparallels(range(60, 85, 5), labels=[1,0,0,0], fontsize=8)

# calculate area loss for each glacier
area_change = (glacier_timeseries - glacier_timeseries.iloc[0])
area_loss = - area_change.iloc[-1]

# create a dataframe for plotting of glacier xy locations and max relative loss
plot_df = pd.DataFrame(columns=['glacier', 'x', 'y', 'loss', 'pct_loss'])
for glacier in glacier_locations:
    lat = glacier_locations[glacier]['lat']
    lon = glacier_locations[glacier]['lon']
    x, y = m(lon, lat)
    loss = area_loss[glacier]
    pct_loss = loss / glacier_timeseries[glacier].iloc[0] * 100
    new_row = {'glacier': glacier, 'x': x, 'y': y, 'loss': loss, 'pct_loss': pct_loss}
    plot_df = pd.concat([plot_df, pd.DataFrame([new_row])], ignore_index=True)

plot_df.set_index('glacier', inplace=True)

sns.scatterplot(data=plot_df, x='x', y='y', size='loss', sizes=(20,200), hue='pct_loss', palette='plasma_r', legend='brief')
plt.title('Glacier Area Loss (1972-2022) by Location')
# customise legend
handles, labels = plt.gca().get_legend_handles_labels()
labels[0], labels[7] = ['Percentage Loss (%)', r'Area Loss ($km^2$)']
plt.legend(bbox_to_anchor=(0.7, 0.95), loc='upper left', handles=handles, labels=labels)
plt.savefig('localised_losses_map')
plt.close()

/tmp/ipykernel_1818/1185047413.py:40: FutureWarning: The behavior of DataFrame concatenation with empty or all-NA entries is deprecated. In a future version, this will no longer exclude empty or all-NA columns when determining the result dtypes. To retain the old behavior, exclude the relevant entries before the concat operation.
  plot_df = pd.concat([plot_df, pd.DataFrame([new_row])], ignore_index=True)
